# Проект

## Регрессия

С 1 октября НДС на бриллианты отменяется, что делает их новым инвестиционным инструментом. Давайте создадим для них модель ценообразования. Dataset содержит характеристики бриллиантов и цены на них с сайта jamesallen (B2C-платформа) по состоянию на 2022-07-01

**Описание характеристик**

* fluor - флюоресценция (свойство драгоценного камня светиться) 
* symmetry - показатель симметрии
* platform - название платформы, на которой был размещен драгоценный камень
* shape - форма
* color - цвет
* clarity - чистота
* cut - качество огранки (может быть только для круглых камней)
* polish - полировка
* id - номер драгоценного камня
* date - дата
* price - цена
* carat - количество каратов
* price_per_carat - цена за карат
* z - длина (диаметр)
* x - ширина
* depth_perc - отношение высоты к ширине
* y - высота

In [61]:
import pandas as pd
import numpy as np

In [62]:
df = pd.read_csv('diamonds.csv')
df.head()

,Unnamed: 0,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
0,135269,NONE,EX,jamesallen,PS,F,SI1,NaN,EX,13870838,202206,12850.0,1.55,8290.32,10.29,6.41,62.0,3.97420
1,48477,MED,EX,jamesallen,RD,H,VVS2,EX,EX,11725253,202207,7510.0,1.02,7362.75,6.41,6.45,62.5,4.03125
2,236786,NONE,EX,jamesallen,EM,H,IF,NaN,EX,14444347,202205,21220.0,2.01,10557.21,8.60,6.37,65.0,4.14050
3,235781,NONE,EX,jamesallen,RD,E,VS2,EX,EX,14438434,202207,8660.0,1.00,8660.00,6.39,6.44,61.4,3.95416
4,277744,NONE,VG,jamesallen,RD,F,VS1,VG,EX,14615276,202206,8480.0,1.00,8480.00,6.28,6.36,62.3,3.96228


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222222 entries, 0 to 222221
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       222222 non-null  int64  
 1   fluor            222207 non-null  object 
 2   symmetry         222218 non-null  object 
 3   platform         222218 non-null  object 
 4   shape            222218 non-null  object 
 5   color            222218 non-null  object 
 6   clarity          222218 non-null  object 
 7   cut              148981 non-null  object 
 8   polish           222218 non-null  object 
 9   id               222222 non-null  int64  
 10  date             222222 non-null  int64  
 11  price            222222 non-null  float64
 12  carat            222222 non-null  float64
 13  price_per_carat  222222 non-null  float64
 14  z                222222 non-null  float64
 15  x                222222 non-null  float64
 16  depth_perc       222222 non-null  floa

Это нужно для того, чтобы предсказать `price_per_carat`.

### Задание 1: Очистка данных
Не все драгоценные камни удается продать в течение месяца, поэтому в таблице есть повторы. Объедините данные по каждому самоцвету: подумайте, как лучше это сделать, какую цену взять.

**Постарайтесь найти аномалии:** возможно, цена на некоторые самоцветы резко меняется (то есть продавец сам не знает, по какой цене их продавать). Также убедитесь, что другие параметры драгоценного камня не меняются.

In [64]:
# Разделим выборку драгоценных камней на две части, встречающиеся 1 раз и которые встречались 2 и более раза
df_one = df.query('id.duplicated(keep=False) == False')
df_three = df.query('id.duplicated(keep=False) == True')

# Удалим аномальные значения и выбросы
df_three = df_three.query('price.diff().abs() <= 0.5 * price.mean()', engine='python').sort_values(by=['id', 'date'])

# Возьмем в качестве цены последнее по времени значение
df_three = df_three.groupby('id').agg('last').reset_index()

# Объединим, чтобы получить результат
df = pd.concat([df_one, df_three]).reset_index(drop=True)

Цены на бриллианты колеблются довольно значительно, попробуйте рассчитать уровень инфляции и скорректировать цены на последний месяц. Следует отметить, что не все цены на бриллианты изменяются синхронно, то есть изменение в отдельных группах может быть разным.

*Определите эти группы и рассчитайте коэффициенты инфляции для каждой из них. Скорректируйте цены на эти коэффициенты и создайте переменную.*

In [67]:
# Определим группы, по которым будет сегментироваться инфляция
categories = ['color', 'clarity', 'carat']  # примеры категорий, включая группу каратов

# Ценовой индекс
df_index = df.groupby(['date'] + categories)[['price']].mean().reset_index()

# Сопоставление групп с максимальной датой
date_max = df_index.date.max()  # последняя дата
df_index = df_index.merge(
    df_index.query('date == @date_max')[categories + ['price']].rename(columns={'price': 'price_per_carat_max'}),
    on=categories, how='outer'
)

# Расчет инфляции
df_index['inflation'] = df_index['price_per_carat_max'] / df_index['price']

# Объединение всех данных в одну таблицу
df_with_inf = df.merge(df_index[['date'] + categories + ['inflation']], on=['date'] + categories, how='left')
df_with_inf.head()

,Unnamed: 0,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation
0,189202,NONE,EX,jamesallen,PS,D,SI2,NaN,EX,14224443,202205,9120.0,1.51,6039.74,10.67,6.23,59.0,3.67570,0.962390
1,76462,NONE,VG,jamesallen,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,0.881565
2,72397,NONE,VG,jamesallen,PS,J,VS1,NaN,VG,12771762,202207,15990.0,2.27,7044.05,11.66,7.26,62.2,4.51572,1.000000
3,269475,MED,EX,jamesallen,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.079663
4,321928,NONE,EX,jamesallen,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.054515


### Задание 2: Модель

Определите функцию потерь (MSE или MAE) и обоснуйте свой выбор. Попробуйте построить baseline. Используйте LabelEncoder для категориальных признаков и **постройте линейную модель**.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LableEncoder
from sklearn.metrics import ...


X = df.drop(columns=["..."])
y = df["..."]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

Теперь попробуйте OHE (One Hot Encoding) или TargetEncoder (сравните их).

Нормализуйте данные.

Поработайте с отсутствующими значениями (обратите внимание, что для 'fluor' значение может быть пропущено или флюоресценция может отсутствовать).

Покажите, насколько вам удалось улучшить результат.

In [ ]:
! pip install category_encoders

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

from typing import Tuple, List


def OHE(df: pd.DataFrame, columns: List[str]) -> Tuple[pd.DataFrame, List[str]]:
    """
    Instead of this function you can use Pandas functionality
    """
    index = df.index
    one = OneHotEncoder(sparse=False, categories='auto')
    ohe = one.fit_transform(df[columns])
    col_names = one.get_feature_names(input_features = columns)
    df = df.drop(columns, axis=1)
    df = df.reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame(ohe, columns=col_names)], axis = 1)
    df = df.set_index(index)
    return (df, col_names)

scaler = StandardScaler() # Помните, что здесь мы выполняем только преобразование

Сравните результаты вашей модели с результатами KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

...

## Деревья решений

Попробуйте выполнить то же задание, но с помощью модели Decision Trees.

In [ ]:
# your code here

## Ансамблевые методы

Попробуйте выполнить то же задание, но с использованием ансамблевых методов (модель случайного леса).

In [ ]:
# your code here

## Заключение

Сравните все модели, выберите лучшую и сделайте вывод о задании в целом.

<Ваш вывод здесь >